In [ ]:
from random import *
import datetime
import pandas as pd

In [ ]:
! gdown --id 1b4E7CnnGS--gbc5ufgWnV7cshMu-Zxlc

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1b4E7CnnGS--gbc5ufgWnV7cshMu-Zxlc
To: /content/May'22.xlsx
100% 340M/340M [00:04<00:00, 82.8MB/s]


In [ ]:
import pandas as pd
df=pd.read_excel("/content/May'22.xlsx")

In [ ]:
# df.head()

In [ ]:
# df.describe()

In [ ]:
# df.info()

In [ ]:
df1 = df.loc[df['Transaction Type'] == 'DEBIT']
cols = ["Payer Account Number ( for DEBIT)","Transaction Date Time","Amount","Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)", "Payer Address"]
df1 = df1[cols]
df1 = df1.set_index('Payer Account Number ( for DEBIT)')
df1.head()

,Transaction Date Time,Amount,Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs),Payer Address
Payer Account Number ( for DEBIT),,,,
100112250233,2022-05-01 00:01:00,5050.00,201007963835,7417071791@ybl
100210076350,2022-05-01 00:01:12,120.00,918020110872063,8650003712@paytm
101010048216,2022-05-01 00:01:13,2570.00,40049975767,ranavishwa@ibl
100910511619,2022-05-01 00:01:20,550.00,167101511293,8445554471@ybl
100310148049,2022-05-01 00:01:50,704.28,19744201000007,9654465184@paytm


In [ ]:
def getRecent(sdf):
  try:
    return sdf["Transaction Date Time"].iloc[-1]
  except:
    return sdf["Transaction Date Time"]
def getRecentAcc_no(sdf):
  try:
    return sdf["Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)"].iloc[-1]
  except:
    return sdf["Payee Account Number (CREDIT txn and mobile nuber to fetch from cbs)"]
def getAddress(sdf):
  d = dict()
  try:
    max=0
    for i in sdf['Payer Address']:
      upi = i.split('@')[1]
      try:
        d[upi]+=1
        max=max(d[upi],max)
      except:
        d[upi]=1
        max=max(d[upi],max)
  except:
    pass
  d["max_recent_freq"]=max
  return str(d)
def TimeSlot(sdf, start, end):
    ct = 0
    try:
      for i in sdf['Transaction Date Time']:
        time = i.time()
        if time>=start and time<=end:
          ct += 1
    except:
      pass
    return ct
def AmountSlot(sdf, minimum, maximum):
  ct = 0
  try:
    for i in sdf['Amount']:
      if i>=minimum and i<=maximum:
        ct += 1
  except:
    pass
  return ct

In [ ]:
def getData(sdf, unique_acc):
  res = [unique_acc]
  
  maxi=0
  start = datetime.time(0, 0, 0)
  end = datetime.time(3, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(4, 0, 0)
  end = datetime.time(7, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(8, 0, 0)
  end = datetime.time(11, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(12, 0, 0)
  end = datetime.time(15, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  start = datetime.time(16, 0, 0)
  end = datetime.time(19, 59, 59)
  res.append(TimeSlot(sdf, start, end))
  maxi=max(maxi,TimeSlot(sdf, start, end))

  start = datetime.time(20, 0, 0)
  end = datetime.time(23, 59, 59)
  v=TimeSlot(sdf, start, end)
  maxi=max(maxi,v)
  res.append(v)

  res.append(maxi)
  maxi=0

  for i in range(0, 100000, 5000):
    minimum = i
    maximum = i+4999
    v=AmountSlot(sdf, minimum, maximum)
    maxi=max(maxi,v)
    res.append(v)

  res.append(maxi)

  res.append(getAddress(sdf))
  res.append(getRecent(sdf))
  res.append(getRecentAcc_no(sdf))

  return res

In [ ]:
ct=0
mdf = pd.DataFrame(columns=['Payer Account Number', 'TimeSlot1', 'TimeSlot2', 'TimeSlot3', 'TimeSlot4', 'TimeSlot5', 'TimeSlot6','maxtimefreq', 'Amount1', 'Amount2', 'Amount3', 'Amount4', 'Amount5', 'Amount6', 'Amount7', 'Amount8', 'Amount9', 'Amount10', 'Amount11', 'Amount12', 'Amount13', 'Amount14', 'Amount15', 'Amount16', 'Amount17', 'Amount18', 'Amount19', 'Amount20','maxamountfreq', 'UPI', 'Recent Transaction', 'Last Payee Account Number'])

for unique_acc in df1.index.unique():
  sdf = df1.loc[unique_acc]
  mdf.loc[ct] = getData(sdf, unique_acc)
  ct += 1

In [ ]:
mdf.head()

,Payer Account Number,TimeSlot1,TimeSlot2,TimeSlot3,TimeSlot4,TimeSlot5,TimeSlot6,maxtimefreq,Amount1,Amount2,...,Amount15,Amount16,Amount17,Amount18,Amount19,Amount20,maxamountfreq,UPI,Recent Transaction,Last Payee Account Number
0,100112250233,1,0,0,2,6,0,6,3,2,...,0,0,0,0,0,0,3,"{'ybl': 1, 'max_recent_freq': 0}",2022-05-13 16:34:50,50200052842753
1,100210076350,1,0,0,3,0,1,3,4,1,...,0,0,0,0,0,0,4,"{'paytm': 1, 'max_recent_freq': 0}",2022-05-27 13:16:45,MC100000564441
2,101010048216,2,0,1,2,3,2,3,9,1,...,0,0,0,0,0,0,9,"{'ibl': 1, 'max_recent_freq': 0}",2022-05-31 18:18:18,60225609146
3,100910511619,5,0,4,2,10,45,45,59,1,...,0,0,0,0,0,0,59,"{'ybl': 1, 'max_recent_freq': 0}",2022-05-30 23:33:23,496002010055220
4,100310148049,1,0,1,7,2,1,7,12,0,...,0,0,0,0,0,0,12,"{'paytm': 1, 'max_recent_freq': 0}",2022-05-30 18:38:37,19744201000007


In [ ]:
def Time(sdf):
  d = dict()
  base = 'TimeSlot'
  times = []
  for i in range(1, 7, 1):
    time = sdf[base+str(i)]
    times.append(time)
  
  times.sort(reverse=True)

  res = []
  
  for i in range(0, 6):
    diff = times[0]-times[i]
    res.append(diff)
  
  return str(res)

In [ ]:
sdf = mdf.iloc[10]
Time(sdf)

'[0, 3, 4, 5, 6, 12]'

In [ ]:
mdf.iloc[10]

Payer Account Number                             101210511282
TimeSlot1                                                   9
TimeSlot2                                                   2
TimeSlot3                                                  11
TimeSlot4                                                   8
TimeSlot5                                                  14
TimeSlot6                                                  10
maxtimefreq                                                14
Amount1                                                    35
Amount2                                                    12
Amount3                                                     4
Amount4                                                     1
Amount5                                                     1
Amount6                                                     1
Amount7                                                     0
Amount8                                                     0
Amount9 

In [ ]:
mdf = mdf.set_index('Payer Account Number')

In [ ]:
def get_score(acc_no,amount,payee_acc_num,payer_address):
  payer_address=payer_address.split('@')[1]
  #try and catch block to check whether the payer account number is present in our database or not
  tdf=None
  try:
    tdf=mdf.loc[acc_no]
  except Exception as e:
    print("Account number is not present in the database")
    return 299

  score=0

  
  #To generate score for frequency corresponding to the given payee account number
  last=tdf["Recent Transaction"]
  last=str(last).split('.')[0]
  last=datetime.datetime.strptime(str(last), '%Y-%m-%d %H:%M:%S')
  last_payee=tdf["Last Payee Account Number"]
  now=datetime.datetime.now()
  curr_time=now.time()
  d=now-last
  if d.days==0:
    time_diff=d.seconds//60
    if time_diff<=4 and payee_acc_num==last_payee:
      score=25
      print("Last transaction was made to the same account number within 4 minutes!!!")
    else:
      score=2.25

  #To generate the score for payer address
  upi=eval(tdf["UPI"])
  if payer_address in upi:
    v=upi.get(payer_address)
    maximum=upi.get('max_recent_freq')
    s=10.0-((10.0/maximum)*v)
    score+=s
  else:
    print("Payer address is not present in the database")
    score+=10.0 #generating very high score because payer address is not present corresponding to the payer account number
  
  #To generate score for time slot
  timeslot="TimeSlot"
  now=datetime.datetime.now()
  curr_time=now.time()
  if curr_time>=datetime.time(0, 0, 0) and curr_time<=datetime.time(3, 59, 59):
    timeslot="TimeSlot1"
  elif curr_time>=datetime.time(4, 0, 0) and curr_time<=datetime.time(7, 59, 59):
    timeslot="TimeSlot2"
  elif curr_time>=datetime.time(8, 0, 0) and curr_time<=datetime.time(11, 59, 59):
    timeslot="TimeSlot3"
  elif curr_time>=datetime.time(12, 0, 0) and curr_time<=datetime.time(15, 59, 59):
    timeslot="TimeSlot4"
  elif curr_time>=datetime.time(16, 0, 0) and curr_time<=datetime.time(19, 59, 59):
    timeslot="TimeSlot5"
  else:
    timeslot="TimeSlot6"
  
  timeslot1=tdf[timeslot]
  maximum=tdf["maxtimefreq"]
  s=10.0-((10.0/maximum)*timeslot1)
  score+=s
  if timeslot1==0:
    print("Out of normal transaction timing habit!!!")
  
  #To generate score for amount slot
  amountslot="Amount"
  k=0
  for i in range(0, 100000, 5000):
    minimum = i
    maximum = i+4999
    k=k+1
    if amount>=minimum and amount<=maximum:
      amountslot=amountslot+str(k)
      break
  amountslot1=tdf[amountslot]
  maximum=tdf["maxamountfreq"]
  s=10.0-((10.0/maximum)*amountslot1)
  score+=s
  if amountslot1==0:
    print("Out of normal transaction amount habit!!!")

  # updating the value of recent transaction date time and payee account number
  tdf["Recent Transaction"]=str(now)
  tdf["Last Payee Account Number"]=payee_acc_num
  mdf.loc[acc_no]=tdf

  return score


In [ ]:
print(get_score(100112250233,250,20148987029,"8130528356@hbd"))

Payer address is not present in the database
Out of normal transaction timing habit!!!
20.0


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [ ]:
ab=mdf.loc[100112250233]
print(ab)

TimeSlot1                                                   1
TimeSlot2                                                   0
TimeSlot3                                                   0
TimeSlot4                                                   2
TimeSlot5                                                   6
TimeSlot6                                                   0
maxtimefreq                                                 6
Amount1                                                     3
Amount2                                                     2
Amount3                                                     2
Amount4                                                     0
Amount5                                                     0
Amount6                                                     1
Amount7                                                     1
Amount8                                                     0
Amount9                                                     0
Amount10